# Schematic Generators

## What you will learn
* How to write schematic generators
* Write a schematic generator for a source follower

## Schematic Generation Flow
1. Create schematic in virtuoso
2. Import schematic from virtuoso to Python
3. Implement schematic design method
4. Use BAG to create new instances of the schematic

## bag_libs.def file
```
BAG_prim $BAG_TECH_CONFIG_DIR/DesignModules
logic_templates laygo/generators/logic/BagModules
adc_sar_templates BagModules
clk_dis_templates BagModules
cds_ff_mpt_microtemplates_dense BagModules
bag_testbenches_ec $BAG_WORK_DIR/bag_testbenches_ec/BagModules
demo_templates $BAG_WORK_DIR/BAG_XBase_demo/BagModules
```
* The "cds.lib" file for BAG
* Each schematic library in virtuoso is converted to a Python package
* bag_libs.def lists locations to save/load schematic generators

## CS Amplifier Schematic Generator
```python
class demo_templates__amp_cs(Module):

    # list of schematic parameters
    param_list = ['lch', 'w_dict', 'intent_dict', 'fg_dict', ]

    def __init__(self, bag_config, parent=None, prj=None, **kwargs):
        Module.__init__(self, bag_config, yaml_file, parent=parent,
                        prj=prj, **kwargs)
        # initialize self.parameters dictionary
        for par in self.param_list:
            self.parameters[par] = None
```
* BAG_XBase_demo/BagModules/demo_templates/amp_cs.py
* Schematic generators are subclass of Module
* self.parameters is a dictionary from parameter names to values

## The Design Method
```python
def design(self, lch=18e-9, w_dict=None, intent_dict=None, fg_dict=None):
    # populate self.parameters dictionary
    local_dict = locals()
    for name in self.param_list:
        if name not in local_dict:
            raise ValueError('Parameter %s not specified.' % name)
        self.parameters[name] = local_dict[name]
```
* design() method declares parameter names and assign default values
* The first five lines populates self.parameters dictionary

## Setting Transistor Parameters
```python
# set transistor parameters
self.instances['XP'].design(w=wp, l=lch, intent=intentp, nf=fg_load)
self.instances['XPD'].design(w=wp, l=lch, intent=intentp, nf=fg_dump)
self.instances['XN'].design(w=wn, l=lch, intent=intentn, nf=fg_amp)

if len(fg_dumn_list) == 1:
    self.instances['XND'].design(w=wn, l=lch, intent=intentn, nf=fg_dumn_list[0])
```
* self.instances is a dictionary of all instances in this schematic, which are Module objects
* BAG_prim transistors have design() method that takes w, l, intent, and nf, and will set the transistor parameters accordingly

## Dummies and array_instance()
```python
else:
    # we have two types of dummies.  Use array_instance to add a new
    # dummy transistor.
    name_list = ['XND0', 'XND1']
    term_list = [{}, dict(D='vout')]
    self.array_instance('XND', name_list, term_list=term_list)
    self.instances['XND'][0].design(w=wn, l=lch, intent=intentn, nf=fg_dumn_list[0])
    self.instances['XND'][1].design(w=wn, l=lch, intent=intentn, nf=fg_dumn_list[1])
```
* Recall that for CS amplifier, dummies have to change depending on (fg_load-fg_amp) % 4
* self.array_instance() convert a single instance to a list of instances
    * Used here to add a new dummy transistor

## SF Schematic Exercise
* Open demo_templates/amp_sf in virtuoso
* Put in the non-dummy transistors and proper connections
    * Named the amplifying transistor XAMP, then bias transistor XBIAS
* See schematic for amp_sf_soln if you're stuck
* Note: don't move pin locations, use stub connection by name
* After completing the schematic, run the following cell which will update the netlist associated with amp_sf

In [2]:
import sys
sys.path.append('BAG_XBase_demo/demo_scripts')
sys.path.append('BAG_XBase_demo/xbase_demo/demo_layout')
import bootcamp_demo as d
#import core
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
# noinspection PyUnresolvedReferences,PyCompatibility
from builtins import *

from bag.layout.routing import TrackID
from bag.layout.template import TemplateBase

from abs_templates_ec.analog_core import AnalogBase

# create BagProject object
local_dict = locals()
if 'bprj' in local_dict:
    print('using existing BagProject')
    bprj = local_dict['bprj']
else:
    print('creating BagProject')
    bprj = d.BagProject()
    
bprj.import_design_library('demo_templates')

creating BagProject


## Implement Schematic Generator
* Fill in the missing parts of the following cell
* When finished, run the cell below it to run the flow
* LVS should pass and characterization should finish successfully
* The solution is listed at the bottom
* NOTE: whenever you change the schematic in virtuoso, you have to import the schematic to Python again

In [5]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
# noinspection PyUnresolvedReferences,PyCompatibility
from builtins import *

import os
import pkg_resources

from bag.design import Module


yaml_file = pkg_resources.resource_filename(__name__, os.path.join('netlist_info', 'amp_sf.yaml'))


# noinspection PyPep8Naming
class demo_templates__amp_sf(Module):
    """Module for library demo_templates cell amp_sf.
    Fill in high level description here.
    """

    param_list = ['lch', 'w_dict', 'intent_dict', 'fg_dict', ]

    def __init__(self, bag_config, parent=None, prj=None, **kwargs):
        Module.__init__(self, bag_config, yaml_file, parent=parent, prj=prj, **kwargs)
        for par in self.param_list:
            self.parameters[par] = None

    def design(self, lch=18e-9, w_dict=None, intent_dict=None, fg_dict=None):
        """To be overridden by subclasses to design this module.
        This method should fill in values for all parameters in
        self.parameters.  To design instances of this module, you can
        call their design() method or any other ways you coded.
        To modify schematic structure, call:
        rename_pin()
        delete_instance()
        replace_instance_master()
        reconnect_instance_terminal()
        restore_instance()
        array_instance()
        """
        local_dict = locals()
        for name in self.param_list:
            if name not in local_dict:
                raise ValueError('Parameter %s not specified.' % name)
            self.parameters[name] = local_dict[name]

        w_amp = w_dict['amp']
        w_bias = w_dict['bias']
        intent_amp = intent_dict['amp']
        intent_bias = intent_dict['bias']
        fg_amp = fg_dict['amp']
        fg_bias = fg_dict['bias']

        # TODO: design XAMP and XBIAS transistors
        # related code from amp_cs schematic generator are copied below
        # for reference
        # self.instances['XP'].design(w=wp, l=lch, intent=intentp, nf=fg_load)
        # self.instances['XPD'].design(w=wp, l=lch, intent=intentp, nf=fg_dump)
        # self.instances['XN'].design(w=wn, l=lch, intent=intentn, nf=fg_amp)

        # algorithm for drawing dummies
        fg_dum_list = fg_dict['dum_list']
        num_dummies = len(fg_dum_list)
        name_list = ['XDUM%d' % idx for idx in range(num_dummies)]

        if (fg_amp - fg_bias) % 4 == 0:
            term_list = [{}, {}, dict(D='VDD')]
        else:
            term_list = [{}, {}, dict(D='vout')]

        self.array_instance('XDUM', name_list, term_list=term_list)
        self.instances['XDUM'][0].design(w=w_bias, l=lch, intent=intent_bias, nf=fg_dum_list[0])
        self.instances['XDUM'][1].design(w=w_amp, l=lch, intent=intent_amp, nf=fg_dum_list[1])
        self.instances['XDUM'][2].design(w=w_amp, l=lch, intent=intent_amp, nf=fg_dum_list[2])

    def get_layout_params(self, **kwargs):
        """Returns a dictionary with layout parameters.
        This method computes the layout parameters used to generate implementation's
        layout.  Subclasses should override this method if you need to run post-extraction
        layout.
        Parameters
        ----------
        kwargs :
            any extra parameters you need to generate the layout parameters dictionary.
            Usually you specify layout-specific parameters here, like metal layers of
            input/output, customizable wire sizes, and so on.
        Returns
        -------
        params : dict[str, any]
            the layout parameters dictionary.
        """
        return {}

    def get_layout_pin_mapping(self):
        """Returns the layout pin mapping dictionary.
        This method returns a dictionary used to rename the layout pins, in case they are different
        than the schematic pins.
        Returns
        -------
        pin_mapping : dict[str, str]
            a dictionary from layout pin names to schematic pin names.
        """
        return {}

In [ ]:
d.run_flow(bprj, top_specs, 'amp_sf', demo_templates__amp_sf)

## Solution
```python
class demo_templates__amp_sf_soln(Module):
    """Module for library demo_templates cell amp_sf_soln.
    Fill in high level description here.
    """

    param_list = ['lch', 'w_dict', 'intent_dict', 'fg_dict', ]

    def __init__(self, bag_config, parent=None, prj=None, **kwargs):
        Module.__init__(self, bag_config, yaml_file, parent=parent, prj=prj, **kwargs)
        for par in self.param_list:
            self.parameters[par] = None

    def design(self, lch=18e-9, w_dict=None, intent_dict=None, fg_dict=None):
        """To be overridden by subclasses to design this module.
        This method should fill in values for all parameters in
        self.parameters.  To design instances of this module, you can
        call their design() method or any other ways you coded.
        To modify schematic structure, call:
        rename_pin()
        delete_instance()
        replace_instance_master()
        reconnect_instance_terminal()
        restore_instance()
        array_instance()
        """
        local_dict = locals()
        for name in self.param_list:
            if name not in local_dict:
                raise ValueError('Parameter %s not specified.' % name)
            self.parameters[name] = local_dict[name]

        w_amp = w_dict['amp']
        w_bias = w_dict['bias']
        intent_amp = intent_dict['amp']
        intent_bias = intent_dict['bias']

        fg_amp = fg_dict['amp']
        fg_bias = fg_dict['bias']

        self.instances['XAMP'].design(w=w_amp, l=lch, intent=intent_amp, nf=fg_amp)
        self.instances['XBIAS'].design(w=w_bias, l=lch, intent=intent_bias, nf=fg_bias)

        # design dummies
        fg_dum_list = fg_dict['dum_list']
        num_dummies = len(fg_dum_list)
        name_list = ['XDUM%d' % idx for idx in range(num_dummies)]

        if (fg_amp - fg_bias) % 4 == 0:
            term_list = [{}, {}, dict(D='VDD')]
        else:
            term_list = [{}, {}, dict(D='vout')]

        self.array_instance('XDUM', name_list, term_list=term_list)
        self.instances['XDUM'][0].design(w=w_bias, l=lch, intent=intent_bias, nf=fg_dum_list[0])
        self.instances['XDUM'][1].design(w=w_amp, l=lch, intent=intent_amp, nf=fg_dum_list[1])
        self.instances['XDUM'][2].design(w=w_amp, l=lch, intent=intent_amp, nf=fg_dum_list[2])

    def get_layout_params(self, **kwargs):
        """Returns a dictionary with layout parameters.
        This method computes the layout parameters used to generate implementation's
        layout.  Subclasses should override this method if you need to run post-extraction
        layout.
        Parameters
        ----------
        kwargs :
            any extra parameters you need to generate the layout parameters dictionary.
            Usually you specify layout-specific parameters here, like metal layers of
            input/output, customizable wire sizes, and so on.
        Returns
        -------
        params : dict[str, any]
            the layout parameters dictionary.
        """
        return {}

    def get_layout_pin_mapping(self):
        """Returns the layout pin mapping dictionary.
        This method returns a dictionary used to rename the layout pins, in case they are different
        than the schematic pins.
        Returns
        -------
        pin_mapping : dict[str, str]
            a dictionary from layout pin names to schematic pin names.
        """
        return {}
```